### PROYECTO FINAL: Aplicación de Machine Learning

In [ ]:
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler
from sklearn.linear_model    import Lasso
from sklearn.metrics         import mean_squared_error, r2_score
from sklearn.preprocessing   import MinMaxScaler, Binarizer

import seaborn as sns

#### 5. Desarrollo de modelo de clasificación 

In [ ]:
from sklearn.linear_model import LogisticRegression

Regresión logística

In [ ]:
logit = LogisticRegression()
logit.fit(X_train_std,y_train)

logit_preds = logit.predict(X_test_std)
logit_preds

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [ ]:
acc =round(accuracy_score(y_test,logit_preds),5)
roc_auc =round(roc_auc_score(y_test,logit_preds),5)

In [ ]:
print(f"accuracy: {acc}") ###cantidad de aciertos (verdad pos + verdad neg / Total)
print(f"ROC-AUC: {roc_auc}") ###

accuracy: 0.68618
ROC-AUC: 0.57563


In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

logit = LogisticRegression(solver='saga')

hyperparams_grid={'penalty':['l1','l2','elasticnet',None],
                    'C':[0.01,0.1,1,10],
                    'fit_intercept':[True,False]

}

grid_search = GridSearchCV(estimator=logit,param_grid=hyperparams_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                                            cv=KFold(n_splits=10,shuffle=True,random_state=2023))

grid_search.fit(X_train_std,y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']
roc_aucs = grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores hiperparametros:{grid_search.best_params_}')
print(f'Mejor Métrica:{grid_search.best_score_}')
print(f'Mean Accuracy: {mean_acc}')
print(f'Mean ROC_AUC: {mean_roc_aucs}')


Mejores hiperparametros:{'C': 0.1, 'fit_intercept': True, 'penalty': 'l1'}
Mejor Métrica:0.7023334229338769
Mean Accuracy: 0.6544502998260551
Mean ROC_AUC: 0.6989877233590157


c:\Users\PC2\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
160 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PC2\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PC2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1471, in fit
    raise ValueError(
ValueError: l1_ratio must be between 0 and 1; got (l1_ratio=None)

--------------------------------------------------------------------------------
80 fits failed with the following error

Matriz de confusión: RL

In [ ]:
import pandas as pd

# Obtener las predicciones en el conjunto de prueba
y_pred = grid_search.predict(X_test_std)

# Calcular la matriz de confusión como un DataFrame
confusion_mtx = pd.crosstab(index=y_test, columns=y_pred, rownames=['Observación'], colnames=['Predicción'])

print("Matriz de confusión:")
print(confusion_mtx)


Matriz de confusión:
Predicción     0    1
Observación          
0            870   88
1            356  104


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier()

hyperparams_grid = {'n_estimators':[1000],
                    'criterion':['gini','entropy','log_loss']

}

grid_search = GridSearchCV(estimator=random_forest,param_grid=hyperparams_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                                            cv=KFold(n_splits=10,shuffle=True,random_state=2023))

grid_search.fit(X_train_std,y_train)

accuracies = grid_search.cv_results_['mean_test_accuracy']
roc_aucs = grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores hiperparametros:{grid_search.best_params_}')
print(f'Mejor Métrica:{grid_search.best_score_}')
print(f'Mean Accuracy: {mean_acc}')
print(f'Mean ROC_AUC: {mean_roc_aucs}')

c:\Users\PC2\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PC2\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PC2\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\PC2\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\Users\PC2\anaconda3\lib\site-packages\joblib\paral

Mejores hiperparametros:{'criterion': 'entropy', 'n_estimators': 1000}
Mejor Métrica:0.6988867774556146
Mean Accuracy: 0.6992927767096951
Mean ROC_AUC: 0.6982420798802138


Matriz de confusión: RF

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Obtener las predicciones en el conjunto de prueba
y_pred = grid_search.predict(X_test_std)

# Calcular la matriz de confusión como un DataFrame
confusion_mtx = pd.crosstab(index=y_test, columns=y_pred, rownames=['Observación'], colnames=['Predicción'])

print("Matriz de confusión:")
print(confusion_mtx)

NameError: name 'grid_search' is not defined

In [ ]:
confusion_matrix = pd.crosstab(y_test, y_preds_nb, rownames=["Observación"], colnames=["Predicción"])
# Calcular la precisión
accuracy = (confusion_matrix.values.diagonal().sum()) / confusion_matrix.values.sum()

# Calcular la especificidad
true_negatives = confusion_matrix.values[0, 0]
false_positives = confusion_matrix.values[0, 1]
specificity = true_negatives / (true_negatives + false_positives)

# Calcular la sensibilidad
true_positives = confusion_matrix.values[1, 1]
false_negatives = confusion_matrix.values[1, 0]
sensitivity = true_positives / (true_positives + false_negatives)

# Imprimir los resultados
print("Precisión (Accuracy):", accuracy)
print("Especificidad (Specificity):", specificity)
print("Sensibilidad (Sensitivity):", sensitivity)

Naive Bayes

In [ ]:
from sklearn.naive_bayes  import GaussianNB

clf_Nb = GaussianNB()
clf_Nb

GaussianNB()

In [ ]:
clf_Nb.fit(X_train_std, y_train)

GaussianNB()

In [ ]:
y_preds_nb = clf_Nb.predict(X_test_std)
y_preds_nb

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_preds_nb))

Accuracy: 0.6332863187588152


Matriz de confusión: NB

In [ ]:
print("Matriz de confusión:\n\n",pd.crosstab(y_test, y_preds_nb, rownames=["Observación"], colnames=["Predicción"]))

Matriz de confusión:

 Predicción     0    1
Observación          
0            729  229
1            291  169


In [ ]:

confusion_matrix = pd.crosstab(y_test, y_preds_nb, rownames=["Observación"], colnames=["Predicción"])
# Calcular la precisión
accuracy = (confusion_matrix.values.diagonal().sum()) / confusion_matrix.values.sum()

# Calcular la especificidad
true_negatives = confusion_matrix.values[0, 0]
false_positives = confusion_matrix.values[0, 1]
specificity = true_negatives / (true_negatives + false_positives)

# Calcular la sensibilidad
true_positives = confusion_matrix.values[1, 1]
false_negatives = confusion_matrix.values[1, 0]
sensitivity = true_positives / (true_positives + false_negatives)

# Imprimir los resultados
print("Precisión (Accuracy):", accuracy)
print("Especificidad (Specificity):", specificity)
print("Sensibilidad (Sensitivity):", sensitivity)

Precisión (Accuracy): 0.6332863187588152
Especificidad (Specificity): 0.7609603340292276
Sensibilidad (Sensitivity): 0.3673913043478261
